In [ ]:
import json
import pandas as pd
import requests
from datetime import datetime

def lambda_handler(event, context):
  schema="gans_analysis"
  host="YOUR HOST"
  user="admin"
  password="YOUR PASSWORD"
  port=3306
  con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


  def extract_forecast(cities):
    from datetime import datetime
    import pytz
    #The time of today is used in the last key-list
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    API_key = "YOUR API KEY"
    weathers_json = []
    #Inicialize the key-lists
    weather_dict = {#'city': [],
                "datetime": [],
                'temperature': [],
                'feels_like': [],
                'detailed_outlook': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'information_retrieved_at': []}
    #Create a loop to add all the JSONS into a list
    for i in range(len(cities)):     
        
        weathers_json.append(requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={cities[i]}&appid={API_key}&units=metric").json())
        #For each timeframe from 00:00 to 21:00, extract the info
        for u in range(3, 10):
            #Ad to each key-list of the dictionary a new item
            #weather_dict['city'].append(weathers_json[i]["city"]["name"])
            weather_dict["datetime"].append(weathers_json[i]["list"][u]["dt_txt"])
            weather_dict['temperature'].append(weathers_json[i]["list"][u]["main"]["temp"])
            weather_dict['feels_like'].append(weathers_json[i]["list"][u]["main"]["feels_like"])
            weather_dict['detailed_outlook'].append(weathers_json[i]["list"][u]["weather"][0]["description"])
            #In case the result is null, adds a 0 to avoid error
            try:
                weather_dict['rain'].append(weathers_json[i]['rain']['3h'])
            except:
                weather_dict['rain'].append('0')
            try:
                weather_dict['snow'].append(weathers_json[i]['snow']['3h'])
            except:
                weather_dict['snow'].append('0')
            weather_dict['wind_speed'].append(weathers_json[i]["list"][u]['wind']['speed'])
            weather_dict['information_retrieved_at'].append(now.strftime('%Y-%m-%d %H:%M:%S'))

    return pd.DataFrame(weather_dict)
    
  def arrivals_tomorrow(icaos):
    from datetime import date, timedelta
    from pytz import timezone


    # DETECTING TOMORROW DAY
    from datetime import datetime, timedelta
    # Extracting present day and hour
    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    # Add one day using 'timedelta()'
    tomorrow = (today + timedelta(days=1))
    
    list_for_df = []
   
    #Itirinete through all the icaos introduced, extracting its info into schedule_json
    for icao in icaos:
        times = [["00:00", "11:59"], ["12:00", "23:59"]]
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
            headers = {
                "X-RapidAPI-Key": "YOUR API KEY",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }        
            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_json = response.json()
            
            url2 = f"https://aerodatabox.p.rapidapi.com/airports/icao/{icao}"
            response_js2 = requests.request("GET", url2, headers=headers).json()
            response_norm2 = pd.json_normalize(response_js2)
            
            
            for flight in flights_json.get('arrivals', []):
                flights_dict = {}
                flights_dict['arrival_icao'] = icao
                
                if 'scheduledTimeLocal' in flight['arrival']:
                    flights_dict['arrival_time_local'] = flight['arrival']['scheduledTimeLocal']
                else:
                    continue
                
                flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
                flights_dict['airline'] = flight['airline'].get('name', None)
                
                flights_dict['data_retrieved_on'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')           
                list_for_df.append(flights_dict)
            
    #Converting the dicttionary into a dataframe now, otherwise I cannot format it later
    df = pd.DataFrame(list_for_df)
    #Formating the time
    df["arrival_time_local"] = df["arrival_time_local"].str.replace(r'\+.*$', '', regex=True)
    return df

  def add_flights_sql(icao):


    schema="gans_analysis"   # name of the database you want to use here
    host="YOUR HOST"   #3306     # to connect to your local server
    user="admin"
    password="YOUR PW" # your password!!!!
    port=3306
    connection_details = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    

    flights_df = arrivals_tomorrow([icao])
    flights_df.to_sql('flights', if_exists='append', con=connection_details,index=False)
    
    
  def add_weather_sql(city_id, city):

    schema="gans_analysis"   # name of the database you want to use here
    host="YOUR HOST"   
    user="admin"
    password="YOUR PW" 
    port=3306
    connection_details = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


    weather_df = extract_forecast([city])
    # Get the city_id for the given city
    #city_id = get_city_id(city)
    
    # Add the city_id as a new column to the weather_df DataFrame
    weather_df['city_id'] = city_id
    
    weather_df.to_sql('weather', if_exists='append', con=connection_details,index=False)
    
    
  def update_tables():
    add_weather_sql(1, "Alicante")
    add_weather_sql(2, "Berlin")
    add_weather_sql(3, "Rome")

    add_flights_sql("LEAL")
    add_flights_sql("LIRF")
    add_flights_sql("EDDB")
      
      
  update_tables()


  return {
  'statusCode': 200,
  'body': json.dumps('Hello from Lambda!')}